In [4]:
#=

Dr Keith Reid
Cailleach Computing Ltd
MIT license

Copyright © 2023 <copyright holders>

Permission is hereby granted, free of charge, to any person obtaining a copy of 
this software and associated documentation files (the “Software”), to deal in
the Software without restriction, including without limitation the rights to 
use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of 
the Software, and to permit persons to whom the Software is furnished to do so, 
subject to the following conditions:

The above copyright notice and this permission notice shall be included in all 
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR 
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS 
FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR 
COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER 
IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
=#

# s.0 libraries

using Test

# s.1 structs

struct Element
    l_link::Char
    name::Char
    r_link::Char
end

# s.2 config

function ask_alphabet(place)
    cardinal            = get_cardinal()
    if cardinal ≤ 62
        universal_alphabet  = collect("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890")
        alphabet            = universal_alphabet[1:cardinal]
    else
        "ran out of letters"
    end
    char = alphabet[place]
end

function get_cardinal()
    cardinal::Int64 = 3
    return cardinal
end

function amend(cardinal, place)
    while place < 1
        place = place + cardinal
    end
    while !(1 ≤ place ≤ cardinal)
        place = place - cardinal
    end
    return place
end

function init_links(cardinal::Int64) # Element(ask_alphabet(i),ask_alphabet(i-1),ask_alphabet(i+1))
    links = Set()
    for i in 1:cardinal
        l_place         = amend(cardinal, i-1)
        name_place      = amend(cardinal, i)
        r_place         = amend(cardinal, i+1)
        
        l_link          = ask_alphabet(l_place)        
        name            = ask_alphabet(name_place)
        r_link          = ask_alphabet(r_place)
        
        this_element    = Element(l_link, name, r_link)
        push!(links, this_element)
    end
    return links
end

# s.5 control

function main_dancing_links()
    cardinal = get_cardinal()
    links    = init_links(cardinal)
    
end

main_dancing_links()

Set{Any} with 3 elements:
  Element('a', 'b', 'c')
  Element('b', 'c', 'a')
  Element('c', 'a', 'b')

In [13]:
# s.6 tests

function test_ask_alphabet()
    place   = 3
    name    = ask_alphabet(place)
    @test name == 'c'
    @test typeof(name) == Char
    place   = 1
    name    = ask_alphabet(place)
    @test name == 'a'
    cardinal= 3
    place   = 0
    place   = amend(cardinal, place)
    name    = ask_alphabet(place)
    @test name == 'c'
    println("passed ask alphabet")
end

function test_amend()
    cardinal = 3
    place    = 1
    amended  = amend(cardinal, place)
    @test amended == 1
    @test 1 ≤ amended ≤ cardinal
    cardinal = 3
    place    = 2
    amended  = amend(cardinal, place)
    @test amended == 2
    @test 1 ≤ amended ≤ cardinal
    cardinal = 3
    place    = 3
    amended  = amend(cardinal, place)
    @test amended == 3
    @test 1 ≤ amended ≤ cardinal
    cardinal = 4
    place    = 3
    amended  = amend(cardinal, place)
    @test amended == 3
    @test 1 ≤ amended ≤ cardinal
    cardinal = 3
    place    = 4
    amended  = amend(cardinal, place)
    @test amended == 1
    @test 1 ≤ amended ≤ cardinal
    cardinal = 3
    place    = 5
    amended  = amend(cardinal, place)
    @test amended == 2
    @test 1 ≤ amended ≤ cardinal
    cardinal = 3
    place    = 6
    amended  = amend(cardinal, place)
    @test amended == 3
    @test 1 ≤ amended ≤ cardinal
    cardinal = 3
    place    = 6
    amended  = amend(cardinal, place)
    @test amended == 3
    @test 1 ≤ amended ≤ cardinal
    cardinal = 3
    place    = 7
    amended  = amend(cardinal, place)
    @test amended == 1
    @test 1 ≤ amended ≤ cardinal
    cardinal = 3
    place    = 0
    amended  = amend(cardinal, place)
    @test amended == 3
    @test 1 ≤ amended ≤ cardinal
    cardinal = 3
    place    = -1
    amended  = amend(cardinal, place)
    @test amended == 2
    @test 1 ≤ amended ≤ cardinal
    cardinal = 7
    place    = -1
    amended  = amend(cardinal, place)
    @test amended == 6
    @test 1 ≤ amended ≤ cardinal
    cardinal = 7
    place    = 0
    amended  = amend(cardinal, place)
    @test amended == 7
    @test 1 ≤ amended ≤ cardinal
    println("passed test")
end

function test_init_links()
    cardinal = 3
    links    = init_links(cardinal)
    @test links == Set(Any[Element('a', 'b', 'c'), Element('b', 'c', 'a'), Element('c', 'a', 'b')])
    @test Element('a', 'b', 'c') ∈ links
    @test Element('b', 'c', 'a') ∈ links
    @test Element('c', 'a', 'b') ∈ links
end

function tests()
    println("testing")
    test_amend()
    test_init_links()
    test_ask_alphabet()
    println("passed all tests")
end

tests()

testing
passed test
passed ask alphabet
passed all tests


In [ ]:
#=
12345678901234567890123456789012345678901234567890123456789012345678901234567890
=#